# Calibration

In [ ]:
import cv2
import numpy as np
import glob
import os

def calibrate_intrinsics(image_glob, pattern_size=(9,6), square_size=0.025):
    """
    Calibrate a single camera's intrinsics using checkerboard images.
    
    Args:
        image_glob (str): Glob pattern for calibration images.
        pattern_size (tuple): Number of inner corners per checkerboard row, col.
        square_size (float): Size of a checkerboard square in meters.
        
    Returns:
        ret (float): RMS re-projection error.
        K (ndarray): Camera matrix (3x3).
        dist (ndarray): Distortion coefficients.
        rvecs, tvecs: Extrinsic vectors for each image.
    """
    # Prepare object points (0,0,0), ..., scaled by square_size
    objp = np.zeros((pattern_size[0]*pattern_size[1],3), np.float32)
    objp[:,:2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1,2)
    objp *= square_size

    objpoints, imgpoints = [], []
    images = glob.glob(image_glob)
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)
        if not ret:
            continue
        corners2 = cv2.cornerSubPix(
            gray, corners, (11,11), (-1,-1),
            criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        )
        objpoints.append(objp)
        imgpoints.append(corners2)

    ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(
        objpoints, imgpoints, gray.shape[::-1], None, None
    )
    print(f"Intrinsics RMS error: {ret:.4f}")
    return ret, K, dist, rvecs, tvecs

def stereo_calibrate(
    glob1, glob2, pattern_size, square_size,
    K1, d1, K2, d2,
    flags=cv2.CALIB_FIX_INTRINSIC
):
    """
    Stereo calibrate two cameras that have already been intrinsically calibrated.
    
    Args:
        glob1, glob2: Glob patterns for the two cameras' images.
        pattern_size, square_size: Checkerboard parameters.
        K1, d1, K2, d2: Intrinsics and distortions for camera 1 and 2.
        flags: calibration flags.
        
    Returns:
        R, T, E, F: Rotation, translation, essential and fundamental matrices.
    """
    objp = np.zeros((pattern_size[0]*pattern_size[1],3), np.float32)
    objp[:,:2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1,2)
    objp *= square_size

    objpoints, imgpoints1, imgpoints2 = [], [], []
    imgs1 = sorted(glob.glob(glob1))
    imgs2 = sorted(glob.glob(glob2))
    for f1, f2 in zip(imgs1, imgs2):
        im1 = cv2.imread(f1); gray1 = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
        im2 = cv2.imread(f2); gray2 = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
        r1, c1 = cv2.findChessboardCorners(gray1, pattern_size, None)
        r2, c2 = cv2.findChessboardCorners(gray2, pattern_size, None)
        if not (r1 and r2):
            continue
        c1 = cv2.cornerSubPix(
            gray1, c1, (11,11), (-1,-1),
            (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        )
        c2 = cv2.cornerSubPix(
            gray2, c2, (11,11), (-1,-1),
            (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        )
        objpoints.append(objp)
        imgpoints1.append(c1)
        imgpoints2.append(c2)

    ret, _, _, _, _, R, T, E, F = cv2.stereoCalibrate(
        objpoints, imgpoints1, imgpoints2,
        K1, d1, K2, d2,
        gray1.shape[::-1],
        criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-5),
        flags=flags
    )
    print(f"Stereo RMS error: {ret:.4f}")
    return R, T, E, F

def calibrate_camera_to_world(
    image_glob, world_objpoints=None,
    pattern_size=(9,6), square_size=0.025,
    K=None, dist=None
):
    """
    Solve PnP to get camera-to-world extrinsics using a known world marker.
    
    Args:
        image_glob: Glob for images where the world marker is visible.
        world_objpoints: (N×3) array of 3D points in world coords.
        pattern_size, square_size: if using a checkerboard as your world frame.
        K, dist: intrinsic calibration for this camera.
        
    Returns:
        rvec, tvec: rotation and translation from world to camera.
    """
    if world_objpoints is None:
        wp = np.zeros((pattern_size[0]*pattern_size[1],3), np.float32)
        wp[:,:2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1,2)
        wp *= square_size
        world_objpoints = wp

    for fname in glob.glob(image_glob):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)
        if not ret:
            continue
        corners2 = cv2.cornerSubPix(
            gray, corners, (11,11), (-1,-1),
            (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        )
        ret, rvec, tvec = cv2.solvePnP(world_objpoints, corners2, K, dist)
        if ret:
            proj, _ = cv2.projectPoints(world_objpoints, rvec, tvec, K, dist)
            err = np.linalg.norm(corners2.reshape(-1,2) - proj.reshape(-1,2)) / len(proj)
            print(f"SolvePnP reprojection error: {err*1e3:.2f} mm")
            return rvec, tvec
    raise RuntimeError("Marker not detected in any image")

if __name__ == "__main__":
    # 1) Intrinsics
    _, K1, d1, _, _ = calibrate_intrinsics("calib_images/cam1/*.png")
    _, K2, d2, _, _ = calibrate_intrinsics("calib_images/cam2/*.png")
    
    # 2) Stereo extrinsics
    R, T, E, F = stereo_calibrate(
        "calib_images/cam1/*.png", "calib_images/cam2/*.png",
        pattern_size=(9,6), square_size=0.025,
        K1=K1, d1=d1, K2=K2, d2=d2
    )
    
    # 3) Camera-to-world
    rvec1, tvec1 = calibrate_camera_to_world(
        "marker_images/cam1/*.png", None,
        pattern_size=(9,6), square_size=0.025,
        K=K1, dist=d1
    )
    rvec2, tvec2 = calibrate_camera_to_world(
        "marker_images/cam2/*.png", None,
        pattern_size=(9,6), square_size=0.025,
        K=K2, dist=d2
    )
    
    print("Calibration complete.")


# Pose Estimation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def back_project(coord, camera_number):
    """
    Back-project a 3D point to 2D pixel coordinates in the image plane.
    
    Args:
        coord: 3D point in world coordinates.
        camera_number: 0 for left camera, 1 for right camera.
        
    Returns:
        pixel_coord: 2D pixel coordinates in the image plane.
    """
    # Camera intrinsics and extrinsics (example values)
    K = np.array([[1000, 0, 320],
                  [0, 1000, 240],
                  [0, 0, 1]])
    R = np.eye(3) if camera_number == 0 else np.array([[0, -1, 0], [1, 0, 0], [0, 0, 1]])
    T = np.array([0.5, 0, 0]) if camera_number == 1 else np.array([0, 0, 0])
    
    # Project the point
    coord_homogeneous = np.append(coord, 1) # Convert to homogeneous coordinates
    pixel_coord_homogeneous = K @ (R @ coord_homogeneous + T)
    pixel_coord = pixel_coord_homogeneous[:2] / pixel_coord_homogeneous[2]
    
    return pixel_coord

# 1) Create lookup table for voxel grid
Nx, Ny, Nz = 20, 20, 20
origin = np.array([0, 0, 0])
voxel_size = np.array([0.01, 0.01, 0.01])
lookup_table = np.zeros((Nx, Ny, Nz, 2, 2), dtype=np.float32)
for x in range(Nx):
    for y in range(Ny):
        for z in range(Nz):
            coord = origin + np.array([x, y, z]) * voxel_size
            for camera_number in range(2):
                pixel_coord = back_project(coord, camera_number)
                lookup_table[x, y, z, camera_number] = np.reshape(pixel_coord, 2)

In [ ]:
# 2) Obtain masks for left and right cameras
# TODO: Replace with actual image loading and processing
mask_0 = np.zeros((480, 640), dtype=np.uint8)
mask_1 = np.zeros((480, 640), dtype=np.uint8)

# 3) Voxel carving: keep (x,y,z) if both silhouettes agree
volume = np.zeros((Nx, Ny, Nz), dtype=np.uint8)
for x in range(Nx):
    for y in range(Ny):
        for z in range(Nz):
            pixel_coord_0 = lookup_table[x, y, z, 0]
            pixel_coord_1 = lookup_table[x, y, z, 1]
            if mask_0[pixel_coord_0[0], pixel_coord_0[1]] and \
               mask_1[pixel_coord_1[0], pixel_coord_1[1]]:
                volume[x, y, z] = 1
                
points = np.argwhere(volume == 1)  # Get indices of carved voxels
                

# 4) Compute a simple centerline: for each z-slice, average x & y
centerline = []
for zk in z:
    slice_pts = points[np.isclose(points[:, 2], zk)]
    if len(slice_pts) > 0:
        cx, cy = slice_pts[:,0].mean(), slice_pts[:,1].mean()
        centerline.append([cx, cy, zk])
centerline = np.array(centerline)

# 5) Estimate tangent at the tip (last two points)
v = centerline[-1] - centerline[-2]
theta = np.arccos(v[2] / np.linalg.norm(v))  # bending magnitude
phi   = np.arctan2(v[1], v[0])              # bending plane

print("Estimated bending angle θ (rad):", theta)
print("Estimated bending plane φ (rad):", phi)

# 6) Plot a 2D x–z view of the carved voxels and the centerline
fig, ax = plt.subplots()
ax.scatter(points[:,0], points[:,2], marker='.', label='voxels')
ax.plot(centerline[:,0], centerline[:,2], label='centerline')
ax.set_xlabel('x')
ax.set_ylabel('z')
ax.legend()
plt.show()


# Create Charuco board

In [ ]:
import cv2, cv2.aruco as aruco
import numpy as np

# print([n for n in dir(cv2.aruco) if n.startswith("DICT")])

# 1. Choose your ArUco dictionary
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

# 2. Create a ChArUco board: 7×5 chessboard squares, each square = 0.04 m, marker length = 0.03 m
charuco_board = aruco.CharucoBoard(
    (11, 8),
    squareLength=0.015, markerLength=0.011,
    dictionary=aruco_dict
)

# 3. Draw & save an image of the board at 200 DPI
img = charuco_board.draw((2000, 1500))
cv2.imwrite("charuco_board.png", img)

AttributeError: 'cv2.aruco.CharucoBoard' object has no attribute 'draw'